In [21]:
"""Data Preprocessing数据预处理"""
import numpy
import os
from PIL import Image

class FACE():
    """
    【人脸数据处理成类】
    输入：
    raw_dir-原始数据所在目录（包含rawdata文件夹和faceDR、faceDS文件）
    变量：
    data-图像数据扁平矩阵(n,d^2)
    image-图像数据矩阵(n,d,d)
    target-性别卷标矩阵（0女，1男）
    label-详细卷标列表
    number-图片号列表
    size-图片数量
    方法：
    find_lb(lable)
    delete(nums)
    index(number)
    """ 
    def __init__(self,raw_dir):
        print('导入数据...')
        with open(raw_dir + '/faceDR','r') as f:
            labels = f.readlines()  # 完整详细卷标
        with open(raw_dir + '/faceDS','r') as f:
            for i in f.readlines():
                labels.append(i)    
        self.number = os.listdir(raw_dir + '/rawdata')  # 存在图片的号码(str)  
        self.label = []  # 整理后的详细卷标
        for i in self.number:  # 删除缺失图像的卷标
            for j in labels:
                if i in j:
                    self.label.append(j)
                    tag0 = 1
                    break
        for i in labels:  
            if 'missing' in i:
                print(i[1:5], '缺失')
        length = len(self.number)
        self.size = length
        self.data = numpy.zeros((length,128**2))
        self.image = numpy.zeros((length,128,128))  # 图像数据矩阵（n*128*128）
        self.target = numpy.zeros(length, dtype='int64')  # 性别卷标矩阵（0女，1男）
        for i in range(length):
            with open(raw_dir + '/rawdata/' + self.number[i],'rb') as f:
                temp = numpy.fromfile(f, dtype=numpy.ubyte)
                if temp.size==128**2:  # 正常大小
                    self.data[i] = temp
                    temp = temp.reshape(128,-1)
                    self.image[i] = temp
                else:  # 需压缩图片
                    im = Image.fromarray(temp.reshape(512,512))
                    temp = im.resize((128, 128), Image.ANTIALIAS)  # 改变尺寸，保持图片高品质
                    print(self.number[i], '已压缩为128*128')        
                    self.image[i] = temp
                if 'female' in self.label[i]:
                    self.target[i] = 0
                elif 'male' in self.label[i]:
                    self.target[i] = 1
                else:
                    self.target[i] = None
        for i in range(len(self.number)):  # 图片号转为int型
            self.number[i] = int(self.number[i])
        print('导入完成！')
                    
                    
    def find_lb(self,label):
        '''find label 返回含有输入卷标的图片号码表'''
        temp = []
        for i in range(self.size):
            if label in self.label[i]:
                temp.append(self.number[i])
        return temp
                       
                       
    def delete(self,nums):
        '''关联性删除输入号码列表对应的数据'''
        if isinstance(nums,list) or isinstance(nums,tuple):
            sum = 0
            not_exist = []
            print('deleting:  ', end='')
            for i in nums:
                if int(i) in self.number:
                    idx = self.number.index(int(i))
                    self.number.pop(idx)
                    self.label.pop(idx)
                    self.data = numpy.delete(self.data,idx,0)
                    self.image = numpy.delete(self.image,idx,0)
                    self.target = numpy.delete(self.target,idx,0)
                    self.size -= 1
                    sum += 1
                    print('%d ' % int(i), end=' ')
                else:
                    not_exist.append(i)
            print('\ndeleted %d/%d images!' % (sum, len(nums)))
            if not_exist:
                print('No.', not_exist, 'don\'t seem to exist')
        else:
            if int(nums) in self.number:
                idx = self.number.index(int(nums))
                self.number.pop(idx)
                self.label.pop(idx)
                self.data = numpy.delete(self.data,idx,0)
                self.image = numpy.delete(self.image,idx,0)
                self.target = numpy.delete(self.target,idx,0)
                self.size -= 1
                print('deleted %d!' % int(nums))
            else:
                print('No.', nums, 'doesn\'t seem to exist')
            
    def index(self,nums):
        '''返回输入图片号在数据中的索引（全部匹配才能正常返回）'''
        if isinstance(nums,list) or isinstance(nums,tuple):
            idx = []
            tag = False
            nums = list(nums)
            for j in nums:
                for i in range(self.size):
                    if int(j)==self.number[i]:
                        idx.append(i)
                        tag = True
                    if i >= self.size-1:
                        if not tag:
                            print('No.', j, 'doesn\'t seem to exist')
                            return
                        tag = False
            return idx
        else:
            for i in range(self.size):
                if int(nums)==self.number[i]:
                    return i
            print('No.', nums, 'doesn\'t seem to exist')


In [22]:
import xlrd  # 表格处理
import matplotlib.pyplot as plt  # 绘图
from sklearn.decomposition import PCA  # 降维
from sklearn.preprocessing import StandardScaler  #标准化
from sklearn.model_selection import train_test_split  #数据集划分
from sklearn.neighbors import KNeighborsClassifier  #KNN
from sklearn.pipeline import make_pipeline  # 顺序处理

face = FACE('C:/Users/sunny/Desktop/模式识别/原始数据+matlab')
# print('\nface.data.shape:', face.data.shape,sep='\n')
# print('\nface.image.shape:', face.image.shape,sep='\n')
# print('\nface.target[::200]:', face.target[::200],sep='\n')
# print('\nface.number[::200]:', face.number[::200],sep='\n')
# print('\nface.label[0]:', face.label[0],sep='\n')
# plt.imshow(face.image[0],cmap=plt.cm.gray)
# plt.show()
# help(face)

导入数据...
1228 缺失
1232 缺失
1808 缺失
4056 缺失
4135 缺失
4136 缺失
5004 缺失
2412 已压缩为128*128
2416 已压缩为128*128
导入完成！


In [23]:
# 删除异常数据
def Del_unusual(xlsdir='C:/Users/sunny/Desktop/noise_collection.xls', sheets=[]):
    '''输入表格路径、工作表名名称和列号（从第二行开始）循环列表，删除face中对应图号的数据'''
    workbook = xlrd.open_workbook(xlsdir)
    del_nums = []   
    for i in sheets:
        sheet = workbook.sheet_by_name(i[0])
        for j in i[1:len(i)]:
            del_nums += [i for i in sheet.col_values(j,1) if i]  # 第2列有数行
    for i in range(len(del_nums)):
        del_nums[i] = int(del_nums[i])
    print('del_nums:', del_nums)
    face.delete(del_nums)


del_clms = [ ['异常汇总', 2, 3]]
Del_unusual(sheets=del_clms)

del_nums: [3883, 2099, 2100, 2101, 2104, 2105, 2106, 3283, 3860, 3861, 4125, 4146, 4237, 4267, 4295, 4335, 4498, 4566, 4679, 4710, 4779, 4992, 5113, 4355, 4356, 4357, 4358, 4359, 2102, 2103, 3862, 4354, 4429, 4452, 3862, 4637, 5076]
deleting:  3883  2099  2100  2101  2104  2105  2106  3283  3860  3861  4125  4146  4237  4267  4295  4335  4498  4566  4679  4710  4779  4992  5113  4355  4356  4357  4358  4359  2102  2103  3862  4354  4429  4452  4637  5076  
deleted 36/37 images!
No. [3862] don't seem to exist


In [24]:
print(face.size)
import numpy as np
np.array(face.target).sum()

3957


2391

In [25]:
print('\n训练中...')
std=StandardScaler()  #标准化
pca = PCA(n_components=75)  # 降维（16384 -> 75）
knn = KNeighborsClassifier()  #KNN
k=5
clf = make_pipeline(std, pca, knn)  # 顺序处理（先标准化数据，再降维，最后用KNN训练/测试）（可以视为自动化分类器，有KNN的所有属性）
Xtrain, Xtest, ytrain, ytest = train_test_split(face.data, face.target,test_size=0.25,random_state=2)  #数据集划分（训练测试3：1）
score = clf.fit(Xtrain, ytrain).score(Xtest,ytest) # 训练、测试、计算准确率
print('测试准确率：', score)


训练中...
测试准确率： 0.7797979797979798
